### Prepare Workspace

In [2]:
import azureml.core
from azureml.core import Workspace
from azureml.core import Experiment
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
import os

from azureml.train.estimator import Estimator
from azureml.train.hyperdrive import RandomParameterSampling, BanditPolicy, HyperDriveRunConfig, PrimaryMetricGoal
from azureml.train.hyperdrive import choice, uniform
from azureml.widgets import RunDetails

### Prepare AML Workspace

In [3]:
# load workspace configuration from the config.json file in the current folder.
ws = Workspace.from_config()
print(ws.name, ws.location, ws.resource_group, ws.location, sep = '\t')
experiment_name = 'mxk'
script_folder = './'
exp = Experiment(workspace=ws, name=experiment_name)


vm_size = "STANDARD_NC6"

compute_target = ws.compute_targets[compute_name]
if compute_target and type(compute_target) is AmlCompute:
    print('found compute target. just use it. ' + compute_name)

ds = ws.get_default_datastore()
print(ds.datastore_type, ds.account_name, ds.container_name)

Found the config file in: C:\Users\makayser\Desktop\git\mxk_retinanet\notebook\aml_config\config.json
mak-ml	westeurope	makshared	westeurope
found compute target. just use it. gpucluster
AzureBlob makml9496683038 azureml-blobstore-43aa3424-3674-489b-808b-1e49daacf13c


In [4]:
# ds.upload(src_dir='./bin', target_path='mxk-train', overwrite=True, show_progress=True)

### Prepare Training

In [4]:
os.chdir('../code')

In [5]:
%load_ext autoreload
%autoreload 2

In [6]:
script_params = {
    '--data-dir': ds.path('mxk').as_mount(),
    '--epochs':6, 
#     '--steps':1800,
#     '--fl-gamma':1.5,
#     '--fl-alpha':0.25,
    '--weights': 'model/resnet50_coco_best_v2.1.0.h5',
    '--backbone' : 'resnet50',
    '--annotations':'train_set_v2_retina.csv',
    '--classes':'classes.csv',
    '--val-annotations':'test_set_v2_retina.csv',
    '--no-snapshots':''
}

In [7]:
script_params

{'--data-dir': $AZUREML_DATAREFERENCE_02093635653c4a179ea671bbac25d0da,
 '--epochs': 6,
 '--weights': 'model/resnet50_coco_best_v2.1.0.h5',
 '--backbone': 'resnet50',
 '--annotations': 'train_set_v2_retina.csv',
 '--classes': 'classes.csv',
 '--val-annotations': 'test_set_v2_retina.csv',
 '--no-snapshots': ''}

In [8]:
est = Estimator(source_directory = script_folder,
                compute_target = compute_target,
                script_params = script_params,
                entry_script = "train.py",
                pip_packages = ["tensorflow-gpu==1.12", "keras", "h5py", "progressbar2", "opencv-python-headless"],
                conda_packages=['opencv', 'h5py','mesa-libgl-cos6-x86_64', 'pillow'],
                use_gpu = True)


In [10]:
max_total_runs=20
max_concurrent_runs=4

In [11]:
param_sampling = RandomParameterSampling( {
        "--lr": uniform(1e-6, 1e-04),
        "--batch-size": choice(1, 4, 8, 16, 32),
        "--fl-gamma": choice(0.75, 1, 1.25, 1.5, 1.75, 2, 2.25, 2.5),
        "--fl-alpha": choice(0.25, 0.5, 0.75, 1)
    }
)
# slack: slack allowed with respect to the best performing training run
early_termination_policy = BanditPolicy(slack_factor = 0.15, evaluation_interval=2)#1, delay_evaluation=4) #0.25

In [12]:
hdc = HyperDriveRunConfig(estimator=est,
                          hyperparameter_sampling=param_sampling, 
                          policy=early_termination_policy,
                          primary_metric_name="EAD_Score", 
                          primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                          max_total_runs=max_total_runs,
                          max_concurrent_runs=max_concurrent_runs)

### Submit Training Run

In [19]:
run = exp.submit(config=hdc)

In [20]:
RunDetails(run).show()

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

In [21]:
run

Experiment,Id,Type,Status,Details Page,Docs Page
mxk,mxk_1552252494771,hyperdrive,Running,Link to Azure Portal,Link to Documentation


In [ ]:
run.wait_for_completion(show_output=True)

RunId: mxk_1552252494771


### Cancel Training

In [29]:
# from azureml.core import Run
# Run(exp, run_id='mxk_1552252494771_12').cancel()
# run.cancel()